# Exploration of the LSTM model
## Goal
Understand how the LSTM model works and try to play with it
## Coding
### 0. Import libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import math, json, datetime

### 1. Create a LSTM class to manage operations with the LSTM model

In [2]:

class LSTM:
    PRIOR_DAY_AMOUNT = 50
    
    def __init__(self, symbol):
        self.symbol = symbol
        self.model = self.build_model()
        self.__init_all_data()
        self.load_model()
        
    def __init_all_data(self):
        self.all_data = []
        with open("dataset/%s.json" % self.symbol, 'r') as openfile:
            self.all_data = json.load(openfile)    
        
    def __init_training_data(self):
        # to remove the datetime from the data
        scale, _ = zip(*self.all_data)
        
        self.scaler = MinMaxScaler(feature_range=(0,1))
        scale = np.array(scale)
        stock_price_history = self.scaler.fit_transform(scale.reshape(-1,1)).reshape(-1)
        self.training_data_x = []
        self.training_data_y = []
        
        training_data_len = math.ceil(len(stock_price_history)* 0.8)
        for i in range(self.PRIOR_DAY_AMOUNT, training_data_len):
            self.training_data_x.append(stock_price_history[i-self.PRIOR_DAY_AMOUNT:i])
            self.training_data_y.append(stock_price_history[i])
        
        self.training_data_x, self.training_data_y = np.array(self.training_data_x), np.array(self.training_data_y)
        self.training_data_x = np.reshape(self.training_data_x, (self.training_data_x.shape[0], self.training_data_x.shape[1], 1))
        self.test = self.training_data_x[-1]
        
    def __delete_training_data(self):
        self.training_data_x, self.training_data_y = None, None
    
    def build_model(self):
        model = keras.Sequential()
        
        model.add(keras.layers.LSTM(100, return_sequences=True, input_shape=(self.PRIOR_DAY_AMOUNT, 1)))
        model.add(keras.layers.LSTM(100, return_sequences=False))
        model.add(keras.layers.Dense(25))
        model.add(keras.layers.Dense(1))
        model.compile(optimizer='adam', loss='mean_squared_error')
        model.summary()
        
        return model
    
    def train(self, batch_size=None, epochs=50):
        self.__init_training_data()
        
        if batch_size is None:
            batch_size = (len(self.all_data) // 350) + 1
            
        self.model.fit(self.training_data_x, self.training_data_y, batch_size=batch_size, epochs=epochs)
        self.save_model()
        #self.__delete_training_data()
        
    def save_model(self):
        self.model.save_weights("parameters/%s/" % self.symbol)
    
    def load_model(self):
        try:
            self.model.load_weights("parameters/%s/" % self.symbol)
        except:
            print("Couldn't load the parameters : the parameters file doesn't exist")
            
    """
        Give predicted price on the given date
    """
    def predict(self, date):
        # Check if date is from monday to friday
        date_to_predict = datetime.datetime.strptime(date, "%Y-%m-%d")
        if date_to_predict.weekday() >= 5:
            return None
        
        last_date = self.all_data[-1][1]
        # Generate needed inputs to predict the price at given date
        
        date_to_predict = np.datetime64(date)
        last_date = np.datetime64(last_date)
        
        business_days_count = np.busday_count(last_date, date_to_predict)
        
        prices, _ = zip(*self.all_data)
        prices = list(self.scaler.fit_transform(np.array(prices).reshape(-1,1)).reshape(-1))
        if business_days_count > 0:
            # Feedforward to get answers and filling inputs as needed
            while business_days_count >= 0:
                inp = np.array(prices[-self.PRIOR_DAY_AMOUNT:]).reshape((1,self.PRIOR_DAY_AMOUNT,1))
                prices.append(self.model.predict(inp)[0][0])
                print(prices[-1])
                business_days_count -= 1
                
            return prices[-1]
        else:
            # We got the required data to predict
            inp = np.array(prices[(-self.PRIOR_DAY_AMOUNT + business_days_count - 1):(business_days_count - 1)]).reshape((1,self.PRIOR_DAY_AMOUNT,1))
            return self.model.predict(inp)[0][0]

### 2. Train the LSTM model

In [4]:
model = LSTM("AAPL")
model.train(epochs=10)

model1 = LSTM("AAPL")
model1.train(batch_size=3,epochs=10)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 50, 100)           40800     
                                                                 
 lstm_3 (LSTM)               (None, 100)               80400     
                                                                 
 dense_2 (Dense)             (None, 25)                2525      
                                                                 
 dense_3 (Dense)             (None, 1)                 26        
                                                                 
Total params: 123,751
Trainable params: 123,751
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
264/264 [==============================] - 5s 11ms/step - loss: 4.3724e-06
Epoch 2/10
264/264 [==============================] - 3s 10ms/step - loss: 4.6665e-06
Epoch 3/

### 3. Try to predict future data

In [5]:
# I will try to predict Apple share price for 2023/02/22
date = "2023-02-07"
price = model.predict(date)
print(model.scaler.inverse_transform([[price]]))

print(model1.scaler.inverse_transform([[model1.predict(date)]]))

1/1 [==============================] - 0s 449ms/step
[[142.50472694]]
1/1 [==============================] - 0s 425ms/step
[[135.62764354]]


### 4. Try different config for the training.
For example, we can check 
1. which optimizer predicts the future best
2. which loss function minimize the overall loss of the model
3. modify the amount of layers / amount of node per layer 
4. mix of #1, #2 and #3 with different batch_size and epoch